## 文本嵌入与向量检索
### 任务


- 任务说明：对文本进行编码，并进行语义检索
- 任务要求：
  - 加载文本编码模型
  - 对提问和文档进行编码，并进行检索
- 打卡要求：加载三个编码模型，计算检索结果

### 语义检索流程

语义检索是通过词嵌入和句子嵌入等技术，将文本表示为语义丰富的向量。通过相似度计算和结果排序找到最相关的文档。用户查询经过自然语言处理处理，最终系统返回经过排序的相关文档，提供用户友好的信息展示。语义检索通过深度学习和自然语言处理技术，使得系统能够更准确地理解用户查询，提高检索的准确性和效果。

<img src="img/TASK5/image-20240213231657725-7837421.png" alt="image-20240213231657725" style="zoom: 33%;" />

### 文本编码模型

文本编码模型对于语义检索的精度至关重要。目前，大多数语义检索系统采用预训练模型进行文本编码，其中最为常见的是基于BERT（Bidirectional Encoder Representations from Transformers）的模型，或者使用GPT（Generative Pre-trained Transformer）等。这些预训练模型通过在大规模语料上进行训练，能够捕捉词语和句子之间的复杂语义关系。选择合适的文本编码模型直接影响到得到的文本向量的有效性，进而影响检索的准确性和效果。

编码模型排行榜：https://huggingface.co/spaces/mteb/leaderboard

- M3E

  ```python
  from sentence_transformers import SentenceTransformer
  model = SentenceTransformer('../hugging-face-model/moka-ai/m3e-small/')

  question_sentences = [x['question'] for x in questions]
  pdf_content_sentences = [x['content'] for x in pdf_content]

  question_embeddings = model.encode(question_sentences, normalize_embeddings=True)
  pdf_embeddings = model.encode(pdf_content_sentences, normalize_embeddings=True)

  for query_idx, feat in enumerate(question_embeddings):
      score = feat @ pdf_embeddings.T
      max_score_page_idx = score.argsort()[-1] + 1
      questions[query_idx]['reference'] = 'page_' + str(max_score_page_idx)

  with open('submit.json', 'w', encoding='utf8') as up:
      json.dump(questions, up, ensure_ascii=False, indent=4)
  ```

- BGE

  ```python
  model = SentenceTransformer('../hugging-face-model/BAAI/bge-small-zh-v1.5/')

  # 剩余代码与M3E部分相同
  ```

- BCEmbedding

  ```python
  model = SentenceTransformer("../hugging-face-model/maidalun1020/bce-embedding-base_v1", device='cuda')
  model.max_seq_length = 512

  # 剩余代码与M3E部分相同
  ```

### 文本切分方法

文本的长度是另一个关键因素，影响了文本编码的结果。短文本和长文本在编码成向量时可能表达不同的语义信息。即使两者包含相同的单词或有相似的语义，由于上下文的不同，得到的向量也会有所不同。因此，当在语义检索中使用短文本来检索长文本时，或者反之，可能导致一定的误差。针对文本长度的差异，有些系统采用截断或填充等方式处理，以保持一致的向量表示。

更多阅读资料：

- https://python.langchain.com/docs/modules/data_connection/document_transformers/
- https://chunkviz.up.railway.app/

| 名称           | 分割依据                   | 描述                                                         |
| :------------- | :------------------------- | :----------------------------------------------------------- |
| 递归式分割器   | 一组用户定义的字符         | 递归地分割文本。递归分割文本的目的是尽量保持相关的文本段落相邻。这是开始文本分割的推荐方式。 |
| HTML分割器     | HTML特定字符               | 基于HTML特定字符进行文本分割。特别地，它会添加有关每个文本块来源的相关信息（基于HTML结构）。 |
| Markdown分割器 | Markdown特定字符           | 基于Markdown特定字符进行文本分割。特别地，它会添加有关每个文本块来源的相关信息（基于Markdown结构）。 |
| 代码分割器     | 代码（Python、JS）特定字符 | 基于特定于编码语言的字符进行文本分割。支持从15种不同的编程语言中选择。 |
| Token分割器    | Tokens                     | 基于Token进行文本分割。存在一些不同的Token计量方法。         |
| 字符分割器     | 用户定义的字符             | 基于用户定义的字符进行文本分割。这是较为简单的分割方法之一。 |
| 语义分块器     | 句子                       | 首先基于句子进行分割。然后，如果它们在语义上足够相似，就将相邻的句子组合在一起。 |

对于自然语言，可以推荐使用Token分割器，结合Chunk Size和Overlap Size可以得到不同的切分：

- **Chunk Size（块大小）**：表示将文本划分为较小块的大小。这是分割后每个独立文本块的长度或容量。块大小的选择取决于应用的需求和对文本结构的理解。
- **Overlap Size（重叠大小）**：指相邻两个文本块之间的重叠部分的大小。在切割文本时，通常希望保留一些上下文信息，重叠大小就是控制这种上下文保留的参数。


In [3]:
import re
from tqdm import tqdm
import json
import pdfplumber
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
page_content  = []

In [4]:

def extract_page_text(filepath, max_len=256, overlap_len=100):
    page_content  = []
    pdf =pdfplumber.open(filepath)
    page_count = 0
    # pattern = r'^\d{1,3}'
    for page in tqdm(pdf.pages):
        page_text = page.extract_text().strip()
        raw_text = [text.strip() for text in page_text.split('\n')]
        new_text = '\n'.join(raw_text)
        new_text = re.sub(r'\n\d{2,3}\s?', '\n', new_text)
        # new_text = re.sub(pattern, '', new_text).strip()
        if len(new_text)>10 and '..............' not in new_text:
            page_content.append(new_text)


    cleaned_chunks = []
    i = 0
    all_str = ''.join(page_content)
    all_str = all_str.replace('\n', '')
    while i<len(all_str):
        cur_s = all_str[i:i+max_len]
        if len(cur_s)>10:
            cleaned_chunks.append(Document(page_content=cur_s, metadata={'page':page_count+1}))
        i+=(max_len - overlap_len)

    return cleaned_chunks,page_content

In [5]:
def get_result(model ,result_name ='m3e' ):


    question_sentences = [x['question'] for x in questions]
    pdf_content_sentences = [x  for x in pdf_content]

    question_embeddings = model.encode(question_sentences, normalize_embeddings=True)
    pdf_embeddings = model.encode(pdf_content_sentences, normalize_embeddings=True)

    for query_idx, feat in enumerate(question_embeddings):
        score = feat @ pdf_embeddings.T
        max_score_page_idx = score.argsort()[-1] + 1
        questions[query_idx]['reference'] = 'page_' + str(max_score_page_idx)

    with open(f'{result_name}_submit.json', 'w', encoding='utf8') as up:
        json.dump(questions, up, ensure_ascii=False, indent=4)



In [6]:
questions = json.load(open("./data/questions.json"))
filepath = './data/初赛训练数据集.pdf'
_,pdf_content = extract_page_text(filepath, max_len=256, overlap_len=100)

100%|██████████| 354/354 [00:06<00:00, 51.83it/s]


In [ ]:
#模型下载
from modelscope import snapshot_download
# model_dir = snapshot_download('AI-ModelScope/bge-large-zh-v1.5',cache_dir='/root/code/')
# model_dir = snapshot_download('xrunda/m3e-base',cache_dir='/root/code/')
# model_dir = snapshot_download('maidalun/bce-embedding-base_v1',cache_dir='/root/code/')


In [13]:
model_m3e = SentenceTransformer('xrunda/m3e-base')
get_result(model = model_m3e,result_name ='m3e' )

In [11]:
model_bge = SentenceTransformer('AI-ModelScope/bge-large-zh-v1___5')
get_result(model = model_bge,result_name ='bge' )

/root/.conda/envs/llm/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [ ]:
model_bce = SentenceTransformer('maidalun/bce-embedding-base_v1')
get_result(model = model_bce,result_name ='bce' )

In [14]:
#模型下载
from modelscope import snapshot_download
model_dir = snapshot_download('jinaai/jina-embeddings-v2-base-zh')

Downloading: 100%|██████████| 190/190 [00:00<00:00, 1.23MB/s]
Downloading: 100%|██████████| 1.19k/1.19k [00:00<00:00, 8.19MB/s]
Downloading: 100%|██████████| 117/117 [00:00<00:00, 630kB/s]
Downloading: 100%|██████████| 51.0/51.0 [00:00<00:00, 240kB/s]
Downloading: 100%|██████████| 8.05k/8.05k [00:00<00:00, 27.9MB/s]
Downloading: 100%|██████████| 328k/328k [00:00<00:00, 6.07MB/s]
Downloading: 100%|█████████▉| 612M/612M [01:16<00:00, 8.37MB/s]
Downloading: 100%|██████████| 307M/307M [00:53<00:00, 5.96MB/s] 
Downloading: 100%|██████████| 154M/154M [00:18<00:00, 8.93MB/s] 
Downloading: 100%|██████████| 95.2k/95.2k [00:00<00:00, 4.76MB/s]
Downloading: 100%|██████████| 229/229 [00:00<00:00, 1.50MB/s]
Downloading: 100%|██████████| 307M/307M [00:54<00:00, 5.87MB/s] 
Downloading: 100%|██████████| 31.3k/31.3k [00:00<00:00, 8.80MB/s]
Downloading: 100%|██████████| 99.0/99.0 [00:00<00:00, 752kB/s]
Downloading: 100%|██████████| 280/280 [00:00<00:00, 2.02MB/s]
Downloading: 100%|██████████| 1.94M/1.94

In [19]:
model_jina = SentenceTransformer('jina-embeddings-v2-base-zh')
get_result(model = model_jina,result_name ='jina' )

/root/.conda/envs/llm/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


## 文本切分方法

#### 一般切分

In [2]:
text = "我是一个名为 ChatGLM3-6B 的人工智能助手，是基于清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型开发的。我的目标是通过回答用户提出的问题来帮助他们解决问题。由于我是一个计算机程序，所以我没有实际的存在，只能通过互联网来与用户交流。"
chunks = []
chunk_size = 32

for i in range(0, len(text), chunk_size):
    chunk = text[i:i + chunk_size]
    chunks.append(chunk)


In [4]:
print(chunks)

['我是一个名为 ChatGLM3-6B 的人工智能助手，是基于清华',
 '大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的',
 '语言模型开发的。我的目标是通过回答用户提出的问题来帮助他们解决问',
 '题。由于我是一个计算机程序，所以我没有实际的存在，只能通过互联网',
 '来与用户交流。']

#### 递归字符切分
使用RecursiveCharacterTextSplitter，一般的设置参数为：chunk_size、 chunk_overlap。

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 35, chunk_overlap=0)
text_splitter.create_documents([text])

[Document(page_content='我是一个名为 ChatGLM3-6B 的人工智能助手，是基于清华大学'),
 Document(page_content='KEG 实验室和智谱 AI 公司于 2023'),
 Document(page_content='年共同训练的语言模型开发的。我的目标是通过回答用户提出的问题来帮助他'),
 Document(page_content='们解决问题。由于我是一个计算机程序，所以我没有实际的存在，只能通过互联'),
 Document(page_content='网来与用户交流。')]

#### 字符切分
使用CharacterTextSplitter，一般的设置参数为：chunk_size、 chunk_overlap、separator和strip_whitespace。



In [6]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size = 35, chunk_overlap=0, separator='', strip_whitespace=False)
text_splitter.create_documents([text])

[Document(page_content='我是一个名为 ChatGLM3-6B 的人工智能助手，是基于清华大学 '),
 Document(page_content='KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型开发'),
 Document(page_content='的。我的目标是通过回答用户提出的问题来帮助他们解决问题。由于我是一个计'),
 Document(page_content='算机程序，所以我没有实际的存在，只能通过互联网来与用户交流。')]

与CharacterTextSplitter不同，RecursiveCharacterTextSplitter不需要设置分隔符，默认的几个分隔符如下：

python复制代码"\n\n" - 两个换行符，一般认为是段落分隔符
"\n" - 换行符
" " - 空格
"" - 字符


#### Markdown切分


In [7]:
# 以下是一段md文本
markdown_text = """
**LangChain-Chatchat** 
基于ChatGLM等大语言模型与Langchain等应用框架实现，开源、可离线部署的检索增强生成(RAG)大模型知识库项目。

## 目录
* 介绍
* 解决的痛点
* 快速上手
* 1. 环境配置
* 2. 模型下载
* 3. 初始化知识库和配置文件
* 4. 一键启动
* 5. 启动界面示例
* 联系我们

## 介绍
一种利用langchain思想实现的基于本地知识库的问答应用，目标期望建立一套对中文场景与开源模型支持友好、可离线运行的知识库问答解决方案。
一行命令运行 Docker ：
```shell
docker run -d --gpus all -p 80:8501 registry.cn-beijing.aliyuncs.com/chatchat/chatchat:0.2.7
```
"""



In [8]:
from langchain.text_splitter import MarkdownTextSplitter
splitter = MarkdownTextSplitter(chunk_size = 40, chunk_overlap=0)
splitter.create_documents([markdown_text])


[Document(page_content='**LangChain-Chatchat**'),
 Document(page_content='基于ChatGLM等大语言模型与Langchain等应用框架实现，开源、可离线'),
 Document(page_content='部署的检索增强生成(RAG)大模型知识库项目。'),
 Document(page_content='## 目录\n* 介绍\n* 解决的痛点\n* 快速上手\n* 1. 环境配置'),
 Document(page_content='* 2. 模型下载\n* 3. 初始化知识库和配置文件\n* 4. 一键启动'),
 Document(page_content='* 5. 启动界面示例\n* 联系我们'),
 Document(page_content='## 介绍'),
 Document(page_content='一种利用langchain思想实现的基于本地知识库的问答应用，目标期望建立一套'),
 Document(page_content='对中文场景与开源模型支持友好、可离线运行的知识库问答解决方案。'),
 Document(page_content='一行命令运行 Docker ：\n```shell'),
 Document(page_content='docker run -d --gpus all -p 80:8501'),
 Document(page_content='registry.cn-beijing.aliyuncs.com/chatch'),
 Document(page_content='at/chatchat:0.2.7'),
 Document(page_content='```')]

#### Python代码切分

In [9]:
python_text = """
class Person:
  def __init__(self, name, age):
    self.name = name
    self.age = age

p1 = Person("John", 36)

for i in range(10):
    print (i)
"""

from langchain.text_splitter import PythonCodeTextSplitter
python_splitter = PythonCodeTextSplitter(chunk_size=100, chunk_overlap=0)
python_splitter.create_documents([python_text])


[Document(page_content='class Person:\n  def __init__(self, name, age):\n    self.name = name\n    self.age = age'),
 Document(page_content='p1 = Person("John", 36)\n\nfor i in range(10):\n    print (i)')]

#### 基于内容意图分块

#### NLTKTextSplitter

In [12]:
import nltk

#解析中文
#做中文分词解析，分割符一定要用“.”才可以正确识别解析（“.”后面一定要一个空格）

text1 = '同是风华正茂，怎敢甘拜下风 . 保持学习，保持饥饿'
Juzi_chinese = nltk.sent_tokenize(text)
print(Juzi_chinese)


['我是一个名为 ChatGLM3-6B 的人工智能助手，是基于清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型开发的。我的目标是通过回答用户提出的问题来帮助他们解决问题。由于我是一个计算机程序，所以我没有实际的存在，只能通过互联网来与用户交流。']


In [ ]:

from langchain.text_splitter import NLTKTextSplitter
text_splitter = NLTKTextSplitter(language = 'zh')
docs = text_splitter.split_text(text,)

#### SpacyTextSplitterm


In [13]:
from langchain.text_splitter import SpacyTextSplitter
text_splitter = SpacyTextSplitter(pipeline = 'zh_core_web_sm')
docs = text_splitter.split_text(text)
docs

['我是一个名为 ChatGLM3-6B 的人工智能助手，是基于清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型开发的。\n\n我的目标是通过回答用户提出的问题来帮助他们解决问题。\n\n由于我是一个计算机程序，所以我没有实际的存在，只能通过互联网来与用户交流。']

### 总结
- NLTKTextSplitter、 SpacyTextSplitter两个分词器没有调通
